<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/FSML_Python/chap04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. 識別 ー統計的手法ー

weatherデータでナイーブベイズ識別を行います。

In [1]:
import numpy as np
import pandas as pd
import io
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.naive_bayes import CategoricalNB

表示を見やすくするために少数点以下の出力を3桁に制限しておきます。

In [2]:
%precision 3
np.set_printoptions(precision=3)  # うまくいかないこともある

weatherデータをpandasのDataFrame形式で読み込みます。

In [3]:
weather_nominal = '''
outlook,temperature,humidity,windy,play
sunny,hot,high,FALSE,no
sunny,hot,high,TRUE,no
overcast,hot,high,FALSE,yes
rainy,mild,high,FALSE,yes
rainy,cool,normal,FALSE,yes
rainy,cool,normal,TRUE,no
overcast,cool,normal,TRUE,yes
sunny,mild,high,FALSE,no
sunny,cool,normal,FALSE,yes
rainy,mild,normal,FALSE,yes
sunny,mild,normal,TRUE,yes
overcast,mild,high,TRUE,yes
overcast,hot,normal,FALSE,yes
rainy,mild,high,TRUE,no
'''

In [4]:
df = pd.read_csv(io.StringIO(weather_nominal), sep=',')
df

,outlook,temperature,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


scikit-learnのデータセット形式であるndarrayとして、パターン行列Xと正解yを取り出します。  

In [5]:
X = df.values[:,0:4]  #左から0列目から3列目まで
y = df.values[:,-1]   #右から1列目

特徴はOrdinalEncoderで数値ベクトルに変換します。
OrdinalEncoderはカテゴリ特徴を整数に置き換えます。

In [6]:
enc = OrdinalEncoder()
X_en = enc.fit_transform(X)

In [7]:
enc.categories_

[array(['overcast', 'rainy', 'sunny'], dtype=object),
 array(['cool', 'hot', 'mild'], dtype=object),
 array(['high', 'normal'], dtype=object),
 array([False, True], dtype=object)]

In [8]:
X_en

array([[2., 1., 0., 0.],
       [2., 1., 0., 1.],
       [0., 1., 0., 0.],
       [1., 2., 0., 0.],
       [1., 0., 1., 0.],
       [1., 0., 1., 1.],
       [0., 0., 1., 1.],
       [2., 2., 0., 0.],
       [2., 0., 1., 0.],
       [1., 2., 1., 0.],
       [2., 2., 1., 1.],
       [0., 2., 0., 1.],
       [0., 1., 1., 0.],
       [1., 2., 0., 1.]])

正解は、LabelEncoderでyesとnoをそれぞれ1と0に置き換えます。

In [9]:
le = LabelEncoder()
y_en = le.fit_transform(y)

In [10]:
le.classes_

array(['no', 'yes'], dtype=object)

In [11]:
y_en

array([0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0])

ナイーブベイズ識別器[CategoricalNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html)の学習を行います。デフォルトでラプラス推定を行っています。

In [12]:
clf = CategoricalNB()
clf.fit(X_en, y_en)

CategoricalNB()

学習した統計モデルの中身を見ます。category\_count\_属性では、各特徴の情報が一つの行列で表されて、行方向がクラス、列方向が特徴の値です。データセット中の頻度が単純にカウントされているだけで、この属性にはスムージングの情報は入っていません。

In [13]:
co = clf.category_count_
for i, c in enumerate(co):
  print(df.columns[i])
  for r in c:
    print(r)

outlook
[0. 2. 3.]
[4. 3. 2.]
temperature
[1. 2. 2.]
[3. 2. 4.]
humidity
[4. 1.]
[3. 6.]
windy
[2. 3.]
[6. 3.]


スムージングされた結果は、feature\_log\_prob\_属性に入っています。

In [14]:
fe = clf.feature_log_prob_
for i, f in enumerate(fe):
  print(df.columns[i])
  for r in f:
    print(r)

outlook
[-2.079 -0.981 -0.693]
[-0.875 -1.099 -1.386]
temperature
[-1.386 -0.981 -0.981]
[-1.099 -1.386 -0.875]
humidity
[-0.336 -1.253]
[-1.012 -0.452]
windy
[-0.847 -0.56 ]
[-0.452 -1.012]


outlook特徴を表す0番目の行列から、クラスが yes（1行目）、特徴の値が overcast（0列目）の値を取り出して、スムージングができていることを検算します。yes が9事例、そのうち overcast が4事例なので、$\log\frac{4+1}{9+3}$を求め、その値が上記データと一致していることを確認します。

In [15]:
import math
math.log(5/12)

-0.875

In [16]:
print(f'{fe[0][1][0]:.3f}')

-0.875


判定結果を得るには、predict_probaメソッドを呼び出します。

例として (sunny, hot, high, FALSE) の判定結果を求めます。結果は、\[noの確率, yesの確率\] という要素を持ったndarrayのリストで帰ってきます。

In [17]:
clf.predict_proba(enc.transform([['sunny', 'hot', 'high', False]]))

array([[0.688, 0.312]])

教科書p.68の結果と少し数値が異なるのは、scikit-learnでは事前確率のスムージングを行っていないからです。下記計算で確認してください。

In [18]:
p=(3/12)*(3/12)*(4/11)*(7/11)*(9/14)
n=(4/8)*(3/8)*(5/7)*(3/7)*(5/14)
p/(p+n)

0.312

# ベイジアンネットワーク

ライブラリ [pgmpy](https://pgmpy.org/)を用いてベイジアンネットワークの計算を行います。

In [19]:
!pip install -U pgmpy

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [20]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.estimators import HillClimbSearch, BayesianEstimator

In [21]:
model = BayesianNetwork([('Rain', 'Wet grass')])
cpd_r = TabularCPD(variable='Rain', variable_card=2, values=[[0.6], [0.4]])
cpd_w = TabularCPD(variable='Wet grass', variable_card=2,
                   values=[[0.8, 0.1],
                           [0.2, 0.9]],
                  evidence=['Rain'],
                  evidence_card=[2])
model.add_cpds(cpd_r, cpd_w)
model.check_model()

True

In [22]:
print(cpd_r)

+---------+-----+
| Rain(0) | 0.6 |
+---------+-----+
| Rain(1) | 0.4 |
+---------+-----+


In [23]:
print(cpd_w)

+--------------+---------+---------+
| Rain         | Rain(0) | Rain(1) |
+--------------+---------+---------+
| Wet grass(0) | 0.8     | 0.1     |
+--------------+---------+---------+
| Wet grass(1) | 0.2     | 0.9     |
+--------------+---------+---------+


何も観測されていない状況で「芝が濡れている」(Wet grass=1)の確率を求める

In [24]:
infer = VariableElimination(model)
w_dist = infer.query(['Wet grass'])
print(w_dist)

+--------------+------------------+
| Wet grass    |   phi(Wet grass) |
+==============+==================+
| Wet grass(0) |           0.5200 |
+--------------+------------------+
| Wet grass(1) |           0.4800 |
+--------------+------------------+


「芝が濡れている」ことが観測されたときの「雨が降った」(Rain=1)確率を求める。「雨が降る」事前確率0.4よりもかなり大きくなっていることを確認。


In [25]:
print(infer.query(['Rain'], evidence={'Wet grass': 1}))

+---------+-------------+
| Rain    |   phi(Rain) |
+=========+=============+
| Rain(0) |      0.2500 |
+---------+-------------+
| Rain(1) |      0.7500 |
+---------+-------------+


### ベイジアンネットワークの学習

クラスを表すplayを最初の列に移動


In [26]:
df2 = df.reindex(columns=['play', 'outlook', 'temperature', 'humidity', 'windy'])
df2

,play,outlook,temperature,humidity,windy
0,no,sunny,hot,high,False
1,no,sunny,hot,high,True
2,yes,overcast,hot,high,False
3,yes,rainy,mild,high,False
4,yes,rainy,cool,normal,False
5,no,rainy,cool,normal,True
6,yes,overcast,cool,normal,True
7,no,sunny,mild,high,False
8,yes,sunny,cool,normal,False
9,yes,rainy,mild,normal,False


In [27]:
est = HillClimbSearch(data=df2)
dag = est.estimate()
edges = dag.edges()
nodes = dag.nodes()
print(edges)
print(nodes)

  0%|          | 0/1000000 [00:00<?, ?it/s]

[('play', 'outlook'), ('play', 'humidity'), ('temperature', 'outlook'), ('humidity', 'temperature'), ('humidity', 'outlook'), ('windy', 'outlook')]
['play', 'outlook', 'temperature', 'humidity', 'windy']


学習されたベイジアンネットワーク

bn6.drawio.svg

条件付き確率表(cpd)を表示。2番目のcpdは、親ノードの値の組み合わせが多いので表示が省略されている

In [28]:
model = BayesianNetwork(dag)
model.fit(df2, estimator=BayesianEstimator, prior_type='K2') # cpds を計算
cpds = model.get_cpds()
for cpd in cpds:
    print(cpd, '\n')

+-----------+-------+
| play(no)  | 0.375 |
+-----------+-------+
| play(yes) | 0.625 |
+-----------+-------+ 

+-------------------+--------------------+-----+-------------------+-------------------+
| humidity          | humidity(high)     | ... | humidity(normal)  | humidity(normal)  |
+-------------------+--------------------+-----+-------------------+-------------------+
| play              | play(no)           | ... | play(yes)         | play(yes)         |
+-------------------+--------------------+-----+-------------------+-------------------+
| temperature       | temperature(cool)  | ... | temperature(mild) | temperature(mild) |
+-------------------+--------------------+-----+-------------------+-------------------+
| windy             | windy(False)       | ... | windy(False)      | windy(True)       |
+-------------------+--------------------+-----+-------------------+-------------------+
| outlook(overcast) | 0.3333333333333333 | ... | 0.25              | 0.25              

特徴ベクトル各次元のすべての値が分かっている場合の識別

In [29]:
X_test = pd.DataFrame([['overcast', 'mild', 'high', False]], columns=['outlook', 'temperature', 'humidity', 'windy'])
X_test

,outlook,temperature,humidity,windy
0,overcast,mild,high,False


In [30]:
y_prob = model.predict_probability(X_test)
y_prob

,play_no,play_yes
0,0.540984,0.459016


特徴ベクトルの一部しか分かっていない場合の識別

In [31]:
infer = VariableElimination(model)
w_dist = infer.query(['play'], evidence={'outlook': 'overcast'})
print(w_dist)

+-----------+-------------+
| play      |   phi(play) |
+===========+=============+
| play(no)  |      0.3219 |
+-----------+-------------+
| play(yes) |      0.6781 |
+-----------+-------------+


近似計算の例

In [32]:
from pgmpy.inference import ApproxInference
infer = ApproxInference(model)
w_dist = infer.query(['play'], evidence={'outlook': 'overcast'})
print(w_dist)

  0%|          | 0/10000 [00:00<?, ?it/s]

+-----------+-------------+
| play      |   phi(play) |
+===========+=============+
| play(yes) |      0.6812 |
+-----------+-------------+
| play(no)  |      0.3188 |
+-----------+-------------+


#練習問題

スムージングを行わない方法でナイーブベイズ識別器の学習を行い、weatherデータ中の特定のデータに対する識別結果の確率を求め、その確率計算が正しいことを検算してください。

[解答例](https://github.com/MasahiroAraki/MLCourse/blob/master/Python/04a_statistical.ipynb)

## 参考

* Naive Bayes https://scikit-learn.org/stable/modules/naive_bayes.html

## 練習問題の解答例

[CategoricalNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html) のマニュアルには、スムージングを行わないようにするにはインスタンスを作るときに `set alpha=0, force_alpha=True` とするように書かれていますが、警告が出るので `alpha=1.0e-10` と小さい値にして計算結果に影響が出ないようにしています。


In [33]:
import numpy as np
import pandas as pd
import io
from sklearn import preprocessing
from sklearn.naive_bayes import CategoricalNB

weather_nominal = '''
outlook,temperature,humidity,windy,play
sunny,hot,high,FALSE,no
sunny,hot,high,TRUE,no
overcast,hot,high,FALSE,yes
rainy,mild,high,FALSE,yes
rainy,cool,normal,FALSE,yes
rainy,cool,normal,TRUE,no
overcast,cool,normal,TRUE,yes
sunny,mild,high,FALSE,no
sunny,cool,normal,FALSE,yes
rainy,mild,normal,FALSE,yes
sunny,mild,normal,TRUE,yes
overcast,mild,high,TRUE,yes
overcast,hot,normal,FALSE,yes
rainy,mild,high,TRUE,no
'''
df = pd.read_csv(io.StringIO(weather_nominal), sep=',')

X = df.values[:,0:4]
y = df.values[:,-1]
enc = preprocessing.OrdinalEncoder()
X_en = enc.fit_transform(X)
le = preprocessing.LabelEncoder()
y_en = le.fit_transform(y)

clf = CategoricalNB(alpha=1.0e-10)
clf.fit(X_en, y_en)

X_test = enc.transform([['sunny', 'hot', 'high', False]])
prob = clf.predict_proba(X_test)
print(f'prob1 = {prob}')

p=(2/9)*(2/9)*(3/9)*(6/9)*(9/14)
n=(3/5)*(2/5)*(4/5)*(2/5)*(5/14)
print(f'prob2 = {p/(p+n):.3}')

prob1 = [[0.795 0.205]]
prob2 = 0.205
